In [ ]:
df = pd.read_csv("/kaggle/input/social-media-sentiments-analysis-dataset/sentimentdataset.csv") 
df.head(5)

In [ ]:
import pandas as pd
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Sentiment'], test_size=0.2, random_state=42)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = logreg_model.predict(X_test_tfidf)
print(y_pred)

In [ ]:
report = classification_report(y_test, y_pred, zero_division=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def parse_classification_report(report):
    lines = report.split("\n")
    lines = [line.strip() for line in lines if line.strip
            ()]
    class_names = []
    metrics = []
    for line in lines[2:]:
        parts = line.split()
        if len(parts) > 1 and parts[0].strip() != 'accuracy':
            class_name = ' '.join(parts[:-4])
            class_names.append(class_name)
            class_metrics = [float(part) for part in parts[-4:] if part != 'support']
            metrics.append(class_metrics)
            
    return class_names, metrics

class_names, metrics = parse_classification_report(report)

In [ ]:
metric_names = ['precision', 'recall', 'f1-score']
metric_dict = {metric: [m[i] for m in metrics] for i, metric in enumerate(metric_names)}

In [ ]:
sns.set(style='whitegrid')
plt.figure(figsize=(10, 6))

In [ ]:
for metric_name, values in metric_dict.items():
    plt.barh(class_names, values, label=metric_name)
    
plt.xlabel('Score')
plt.ylabel('Performance Metrics')
plt.legend()
plt.tight_layout()
plt.show()